In [1]:
import json
import os
import warnings

import numpy as np
import pandas as pd


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [2]:
def save_object(obj, name, out_dir='out'):
    file_name = name if name.endswith('.json') else (name + '.json')
    file_path = os.path.join(out_dir, file_name)
    with open(file_path, 'w') as f:
        json.dump(obj, f)

def load_object(name, in_dir='out'):
    file_name = name if name.endswith('.json') else (name + '.json')
    file_path = os.path.join(in_dir, file_name)
    with open(file_path, 'r') as f:
        return json.load(f)

In [3]:
kbr_data_path = '../../data/raw/KBR/'
kbr_dir_path = '1_Ankiety_w_gospodarstwach_domowych/'
kbr_file_path = 'Etap_V_1_1_Ankiety_w_gospodarstwach_domowych_Wroclaw.xlsx'
excel_sheet = 'Wrocław_ankiety+podróże'

data = pd.read_excel(os.path.join(kbr_data_path, kbr_dir_path, kbr_file_path), excel_sheet, header=[0,1], engine='openpyxl')

data.columns = [f'{i}_{j}' for i, j in data.columns]
data = data.iloc[0:14241]

if data.shape[1] == len(set(list(data.columns))):
    print('All column names are unique.')
else:
    print('All column names are NOT unique.')

All column names are unique.


In [4]:
data.head()

,Lp_Lp,IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza),IDENTYFIKACJA ANKIETY_Nr ankietera,IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Godzina wypełnienia kwestionariusza,IDENTYFIKACJA ANKIETY_Nr rejonu komunikacyjnego,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Wprowadzenie opłat za wjazd do centrum dla wszystkich pojazdów,"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dopuszczenie dojazdu do centrum dla pojazdów, które nie zanieczyszczają środowiska (np. pojazdy elektryczne, hybrydowe lub spełniające najwyższe normy spalania)","PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwiązań wspierających kursowanie tramwajów i autobusów, takich jak priorytet przejazdu na skrzyżowaniach oraz wydzielone pasy na jezdniach?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Z jakiego środka transportu zbiorowego korzysta Pan/Pani najczęściej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? \n[tramwaj/autobus],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Dlaczego? [Inne]\n[tramwaj/autobus],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jaka jest Pana/Pani opinia na temat rozwoju systemu kolei aglomeracyjnej na terenie Wrocławia i wokół niego (rozbudowa układu torowego, zwiększenie częstotliwości itd.)?",PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia wygodę jazdy pojazdami komunikacji zbiorowej?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia czystość taboru komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak Pan/Pani ocenia punktualność komunikacji zbiorowej we Wrocławiu?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jak ocenia Pan/Pani efekty dotychczasowych działań związanych z rozbudową systemu rowerowego we Wrocławiu (dróg i parkingów rowerowych)?,PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niekorzystne ustawienia sygnalizacji świetlnej],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak chodników i konieczność poruszania się jezdnią/poboczem/wydeptaną ścieżką],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zły stan nawierzchni chodników],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zastawianie chodników przez parkujące samochody],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niebezpieczne zachowania kierowców],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zagrożenie ze strony rowerzystów poruszających się chodnikami],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [zbyt wysokie krawężniki],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [brak bieżącego utrzymania czystości/odśnieżania],PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wskazać maksymalnie 3 odpowiedzi) [niewystarczająca liczba przejść dla pieszych],"PREFERENCJE DOT. MOBILNOŚCI MIEJSKIEJ_Jakie elementy stanowią dla Pana/Pani największa uciążliwość w codziennych podróżach pieszych po Wrocławiu (można wska

In [5]:
chosen_cols = [
    'IDENTYFIKACJA ANKIETY_ID_LOS (numer kwestionariusza)',
    'IDENTYFIKACJA OSOBY_Nr. osoby w gosp. dom.',
    'IDENTYFIKACJA ANKIETY_Data wypełnienia kwestionariusza',
    'DANE O RESPONDENCIE_Przedział wiekowy',
    'DANE O RESPONDENCIE_Płeć',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Godzina rozpoczęcia podróży\n[gg:mm:ss]',
    'OPIS PODRÓŻY "ŹRÓDŁO"_Z jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca',
    'OPIS PODRÓŻY "CEL"_Do jakiego miejsca [Inne]',
    '23:59:00_motywacje'
]

data_filtered = data[chosen_cols]

data_filtered.columns = [
    'id', 
    'household_person_id',
    'questionnaire_date',
    'age_bracket', 
    'sex',
    'trip_start_time',
    'source', 
    'destination',
    'what_inne',
    'source-destination'
]

data_filtered.fillna(0, inplace=True)
# data_filtered.dropna(inplace=True)
# data_filtered.reset_index(drop=True, inplace=True)

In [6]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination
0,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:00:00,dom,Inne,SKLEP,dom-Inne
1,LOS_67-67-306,1.0,2018-04-25,61 i więcej (emeryci kobiety),Kobieta,09:40:00,Inne,dom,0,Inne-dom
2,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Kobieta,0,0,0,0,-
3,LOS_67-67-109,0.0,2018-04-26,20-24 (wiek studencki),Mężczyzna,0,0,0,0,-
4,LOS_67-67-401,0.0,2018-04-26,61 i więcej (emeryci kobiety),Kobieta,0,0,0,0,-


In [7]:
age_bracket_map_dict = {
    '6-15 (dzieci)': '6-15',
    '16-19 (młodzież)': '16-19',
    '20-24 (wiek studencki)': '20-24',
    '25-44 (młodsi pracownicy)': '25-44',
    '45-60 (starsi pracownicy kobiety)': '45-60',  # '45-60(K)',
    '45-65 (starsi pracownicy mężczyźni)': '45-65',  # '45-65(M)',
    '61 i więcej (emeryci kobiety)': '61-x',  # '61-(K)',
    '66 i więcej (emeryci mężczyźni)': '66-x'  # '66-(M)'
}
data_filtered.replace({'age_bracket': age_bracket_map_dict}, inplace=True)
sex_map_dict = {
    'Kobieta': 'K',
    'Mężczyzna': 'M'
}
data_filtered.replace({'sex': sex_map_dict}, inplace=True)

data_filtered['age_sex_comb'] = data_filtered['age_bracket'] + '_' + data_filtered['sex']
data_filtered['person_id'] = data_filtered['id'] + data_filtered['questionnaire_date'].astype('str') + data_filtered['household_person_id'].astype('str')
data_filtered['trip_start_time'] = data_filtered['trip_start_time'].astype('str')
data_filtered['source'] = data_filtered['source'].str.replace('Inne', 'inne').replace('szkoła', 'szkola')
data_filtered['destination'] = data_filtered['destination'].str.replace('Inne', 'inne').replace('szkoła', 'szkola')
data_filtered['what_inne'] = data_filtered['what_inne'].str.replace('Inne', 'inne').replace('szkoła', 'szkola')
data_filtered['source-destination'] = data_filtered['source-destination'].str.replace('Inne', 'inne', regex=True).replace('szkoła', 'szkola', regex=True)

In [8]:
# bez inne
# data_filtered = data_filtered[
#     (~data_filtered['source-destination'].str.contains('inne')) &
#     (data_filtered['source-destination'] != '-')
# ]

# z inne
data_filtered = data_filtered[
    data_filtered['source-destination'] != '-'
]

In [9]:
data_filtered['trip_start_time'].value_counts()

16:00:00    511
17:00:00    432
15:00:00    416
07:30:00    402
10:00:00    383
           ... 
08:27:00      1
18:18:00      1
09:22:00      1
09:48:00      1
06:55:00      1
Name: trip_start_time, Length: 440, dtype: int64

In [10]:
data_filtered[
    data_filtered['trip_start_time'] == '1899-12-30 00:00:00'
]

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination,age_sex_comb,person_id
11117,LOS_39_39_1412,1.0,2018-10-12,25-44,K,1899-12-30 00:00:00,inne,dom,NaN,inne-dom,25-44_K,LOS_39_39_14122018-10-121.0


In [11]:
data_filtered = data_filtered[
    data_filtered['trip_start_time'] != '1899-12-30 00:00:00'
]

In [12]:
data_filtered = data_filtered.sort_values(by=['person_id', 'trip_start_time'])

In [15]:
data_filtered.head()

,id,household_person_id,questionnaire_date,age_bracket,sex,trip_start_time,source,destination,what_inne,source-destination,age_sex_comb,person_id
4054,LOS_100-100-1305,1.0,2018-06-14,61-x,K,13:05:00,dom,inne,Pętle,dom-inne,61-x_K,LOS_100-100-13052018-06-141.0
4055,LOS_100-100-1305,1.0,2018-06-14,61-x,K,14:30:00,inne,dom,NaN,inne-dom,61-x_K,LOS_100-100-13052018-06-141.0
7911,LOS_100-100-164,1.0,2018-06-22,25-44,K,17:00:00,dom,praca,NaN,dom-praca,25-44_K,LOS_100-100-1642018-06-221.0
7928,LOS_100-100-2281,1.0,2018-06-22,25-44,K,08:00:00,dom,inne,inne,dom-inne,25-44_K,LOS_100-100-22812018-06-221.0
7972,LOS_100-100-2281,1.0,2018-06-22,25-44,K,10:40:00,inne,dom,NaN,inne-dom,25-44_K,LOS_100-100-22812018-06-221.0


In [16]:
data_filtered.groupby('person_id').size().value_counts()

2     4556
4      401
3      366
1      154
6       43
5       41
8        4
7        2
19       1
14       1
10       1
77       1
dtype: int64

In [17]:
# data_filtered['source-destination-agg'] = 
data_filtered[['person_id', 'source-destination']].groupby(
    ['person_id'])['source-destination'].transform(lambda x: ' => '.join(x))

4054       dom-inne => inne-dom
4055       dom-inne => inne-dom
7911                  dom-praca
7928       dom-inne => inne-dom
7972       dom-inne => inne-dom
                  ...          
11679    dom-praca => praca-dom
11680    dom-praca => praca-dom
11681    dom-praca => praca-dom
10947      dom-inne => inne-dom
10948      dom-inne => inne-dom
Name: source-destination, Length: 12597, dtype: object

In [18]:
df_trip_chains = data_filtered.groupby(['age_sex_comb', 'person_id'])['source-destination']\
    .apply(lambda x: ', '.join(x))\
    .reset_index()

df_trip_chains = df_trip_chains.groupby('age_sex_comb')['source-destination'].value_counts().to_frame('count').reset_index()

In [23]:
print(df_trip_chains.shape)
df_trip_chains.head()

(364, 3)


,age_sex_comb,source-destination,count
0,16-19_K,"dom-szkola, szkola-dom",50
1,16-19_K,"dom-inne, inne-dom",8
2,16-19_K,"dom-uczelnia, uczelnia-dom",8
3,16-19_K,"dom-szkola, szkola-dom, dom-inne, inne-dom",4
4,16-19_K,"dom-praca, praca-dom",3


In [22]:
out_dir = './'
file_name = 'trip_chains.csv'

In [24]:
df_trip_chains.to_csv(os.path.join(out_dir, file_name), index=False)

### At this point the file has been manually corrected / prepared for further calculations

In [25]:
in_dir = './'
file_name = 'trip_chains_corrected.csv'

In [26]:
df_trip_chains = pd.read_csv(os.path.join(in_dir, file_name))

print(df_trip_chains.shape)
df_trip_chains.head()

(341, 3)


,age_sex_comb,source-destination,count
0,16-19_K,"dom-szkoła, szkoła-dom",50
1,16-19_K,"dom-inne, inne-dom",8
2,16-19_K,"dom-uczelnia, uczelnia-dom",8
3,16-19_K,"dom-szkoła, szkoła-dom, dom-inne, inne-dom",4
4,16-19_K,"dom-praca, praca-dom",3


In [27]:
# validation
print(
    df_trip_chains[
        (df_trip_chains['age_sex_comb'] == '16-19_K') &
        (df_trip_chains['source-destination'] == 'dom-inne, inne-dom')
    ]
)

print(
    df_trip_chains[
        (df_trip_chains['age_sex_comb'] == '16-19_K') &
        (df_trip_chains['source-destination'] == 'dom-praca, praca-dom')
    ]
)

  age_sex_comb  source-destination  count
1      16-19_K  dom-inne, inne-dom      8
5      16-19_K  dom-inne, inne-dom      1
  age_sex_comb    source-destination  count
4      16-19_K  dom-praca, praca-dom      3
8      16-19_K  dom-praca, praca-dom      1


### Merge same source-destination chains after correction

In [28]:
df_trip_chains = df_trip_chains.groupby(['age_sex_comb', 'source-destination'])['count'].sum().to_frame('count').reset_index()
df_trip_chains = df_trip_chains.sort_values(by=['age_sex_comb', 'count'], ascending=[True, False])
df_trip_chains['source-destination'] = df_trip_chains['source-destination'].str.replace('szkoła', 'szkola', regex=True)

print(df_trip_chains.shape)
df_trip_chains.head()

(234, 3)


,age_sex_comb,source-destination,count
6,16-19_K,"dom-szkola, szkola-dom",50
0,16-19_K,"dom-inne, inne-dom",9
10,16-19_K,"dom-uczelnia, uczelnia-dom",8
3,16-19_K,"dom-praca, praca-dom",4
7,16-19_K,"dom-szkola, szkola-dom, dom-inne, inne-dom",4


### Add probs to source-destination chains grouped by age_sex_comb

In [29]:
df_trip_chains = df_trip_chains.groupby(['age_sex_comb', 'source-destination'])['count'].sum().agg(
    {'count': lambda x: x, 'prob': lambda x: x / x.sum(level=0)}
).unstack(level=0).sort_values(by=['age_sex_comb', 'count'], ascending=[True, False]).reset_index()

df_trip_chains.head()

,age_sex_comb,source-destination,count,prob
0,16-19_K,"dom-szkola, szkola-dom",50.0,0.595238
1,16-19_K,"dom-inne, inne-dom",9.0,0.107143
2,16-19_K,"dom-uczelnia, uczelnia-dom",8.0,0.095238
3,16-19_K,"dom-praca, praca-dom",4.0,0.047619
4,16-19_K,"dom-szkola, szkola-dom, dom-inne, inne-dom",4.0,0.047619


### Add destinations column (list extracted from source-destination chains)

In [30]:
def add_destinations_col(row):
    destinations = []
    sds = row['source-destination'].split(', ')
    for sd in sds:
        destinations.append(sd.split('-')[1])
    return destinations


df_trip_chains['destinations'] = df_trip_chains.apply(add_destinations_col, axis=1)
df_trip_chains.head()

,age_sex_comb,source-destination,count,prob,destinations
0,16-19_K,"dom-szkola, szkola-dom",50.0,0.595238,"[szkola, dom]"
1,16-19_K,"dom-inne, inne-dom",9.0,0.107143,"[inne, dom]"
2,16-19_K,"dom-uczelnia, uczelnia-dom",8.0,0.095238,"[uczelnia, dom]"
3,16-19_K,"dom-praca, praca-dom",4.0,0.047619,"[praca, dom]"
4,16-19_K,"dom-szkola, szkola-dom, dom-inne, inne-dom",4.0,0.047619,"[szkola, dom, inne, dom]"


### Add no_travels column (number of travels)

In [31]:
df_trip_chains['no_travels'] = df_trip_chains['destinations'].apply(lambda x: len(x))
df_trip_chains.head()

,age_sex_comb,source-destination,count,prob,destinations,no_travels
0,16-19_K,"dom-szkola, szkola-dom",50.0,0.595238,"[szkola, dom]",2
1,16-19_K,"dom-inne, inne-dom",9.0,0.107143,"[inne, dom]",2
2,16-19_K,"dom-uczelnia, uczelnia-dom",8.0,0.095238,"[uczelnia, dom]",2
3,16-19_K,"dom-praca, praca-dom",4.0,0.047619,"[praca, dom]",2
4,16-19_K,"dom-szkola, szkola-dom, dom-inne, inne-dom",4.0,0.047619,"[szkola, dom, inne, dom]",4


### Prepare structure for .json format

In [32]:
age_sex_comb_list = df_trip_chains['age_sex_comb'].unique().tolist()
place_type_list = data_filtered['source'].unique().tolist()

travel_chains_dist = {}
for age_sex_comb in age_sex_comb_list:
    travel_chains_dist[age_sex_comb] = {}

for idx, row in df_trip_chains.iterrows():
    age_sex_comb = row['age_sex_comb']
    destinations_list = row['destinations']
    destinations_str = ','.join(destinations_list)
    prob = row['prob']
    # for dest in destinations_l:
    #     if dest not in place_type_list:
    #         print(dest)
    travel_chains_dist[age_sex_comb][destinations_str] = prob

In [33]:
travel_chains_dist['16-19_K']

{'szkola,dom': 0.5952380952380952,
 'inne,dom': 0.10714285714285714,
 'uczelnia,dom': 0.09523809523809523,
 'praca,dom': 0.047619047619047616,
 'szkola,dom,inne,dom': 0.047619047619047616,
 'inne,inne,szkola,inne,inne,dom': 0.011904761904761904,
 'inne,szkola,dom,inne,dom': 0.011904761904761904,
 'praca,dom,inne,dom': 0.011904761904761904,
 'praca,inne,dom': 0.011904761904761904,
 'szkola,inne,dom': 0.011904761904761904,
 'szkola,szkola,inne,inne,inne,inne,dom': 0.011904761904761904,
 'uczelnia,dom,inne,dom': 0.011904761904761904,
 'uczelnia,dom,uczelnia,dom': 0.011904761904761904,
 'uczelnia,praca,dom': 0.011904761904761904}

### Save do .json

In [34]:
out_dir = './'
file_name = 'travel_chains_dist.json'

save_object(
    obj=travel_chains_dist,
    name=file_name,
    out_dir=out_dir
)

___
___
___

### Calculate number of travels distributions (general and grouped by age_sex_comb)

General

In [35]:
df_num_travels = df_trip_chains.groupby(by=['no_travels'])['count'].sum().to_frame('count').reset_index()
count_sum = df_num_travels['count'].sum()
df_num_travels['prob'] = df_num_travels['count'] / count_sum
df_num_travels

,no_travels,count,prob
0,2,4854.0,0.855934
1,3,391.0,0.068947
2,4,358.0,0.063128
3,5,42.0,0.007406
4,6,23.0,0.004056
5,7,2.0,0.000353
6,8,1.0,0.000176


Grouped by age_sex_comb

In [36]:
df_num_travels = df_trip_chains.groupby(['age_sex_comb', 'no_travels'])['count'].sum().agg(
    {'count': lambda x: x, 'prob': lambda x: x / x.sum(level=0)}
).unstack(level=0).reset_index()

df_num_travels['count'] = df_num_travels['count'].astype('int32')
df_num_travels

,age_sex_comb,no_travels,count,prob
0,16-19_K,2,71,0.845238
1,16-19_K,3,3,0.035714
2,16-19_K,4,7,0.083333
3,16-19_K,5,1,0.011905
4,16-19_K,6,1,0.011905
5,16-19_K,7,1,0.011905
6,16-19_M,2,107,0.862903
7,16-19_M,3,8,0.064516
8,16-19_M,4,6,0.048387
9,16-19_M,5,1,0.008065


## Other struff

### Probs for joint probabilities approach

Probs preparation

In [40]:
age_sex_comb_list = df_trip_chains['age_sex_comb'].unique().tolist()
place_type_list = data_filtered['source'].unique().tolist()

new_dest_type_dist = {}
for age_sex_comb in age_sex_comb_list:
    new_dest_type_dist[age_sex_comb] = {}
    for source in place_type_list:
        new_dest_type_dist[age_sex_comb][source] = {}
        for dest in place_type_list:
            new_dest_type_dist[age_sex_comb][source][dest] = 0

In [41]:
for idx, row in df_trip_chains.iterrows():
    age_sex_comb = row['age_sex_comb']
    count = row['count']
    
    for curr_source_idx, curr_source in enumerate(row['destinations']):
        dest_place_type_set = set(row['destinations'][curr_source_idx+1:])
        while dest_place_type_set:
            dpt = dest_place_type_set.pop()
            new_dest_type_dist[age_sex_comb][curr_source][dpt] += int(count)

In [42]:
new_dest_type_dist['16-19_K']

{'dom': {'dom': 8, 'inne': 7, 'uczelnia': 1, 'praca': 0, 'szkola': 0},
 'inne': {'dom': 27, 'inne': 7, 'uczelnia': 0, 'praca': 0, 'szkola': 3},
 'uczelnia': {'dom': 12, 'inne': 1, 'uczelnia': 1, 'praca': 1, 'szkola': 0},
 'praca': {'dom': 7, 'inne': 2, 'uczelnia': 0, 'praca': 0, 'szkola': 0},
 'szkola': {'dom': 59, 'inne': 9, 'uczelnia': 0, 'praca': 0, 'szkola': 1}}

Probs distr

In [43]:
for age_sex_comb, source_place_types in new_dest_type_dist.items():
    for source_place_type, destination_place_types in source_place_types.items():
        count_sum = np.array(list(destination_place_types.values())).sum()
        if count_sum != 0:
            for i in list(new_dest_type_dist[age_sex_comb][source_place_type].keys()):
                new_dest_type_dist[age_sex_comb][source_place_type][i] = new_dest_type_dist[age_sex_comb][source_place_type][i] / count_sum

Save to .json

In [44]:
out_dir = './'
file_name = 'new_dest_type_dist.json'

save_object(
    obj=new_dest_type_dist,
    name=file_name,
    out_dir=out_dir
)